<a href="https://colab.research.google.com/github/nalrob/Datos_Masivos_MCD/blob/main/Extra-Challenge_Ensemble_with_Scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extra-Challenge


Building an Ensemble Learning Model Using Scikit-learn 

In [28]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score

df = pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [7]:
X = df.drop(columns = ['Outcome'])
y = df['Outcome']

In [8]:
from sklearn.model_selection import train_test_split
#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

**1. k-Nearest Neighbors (k-NN)**

In [33]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Medir el tiempo de ejecución
start_time_1 = time.time()

#create new a knn model
knn = KNeighborsClassifier()
#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}
#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=5)
#fit model to training data
knn_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [34]:
#save best model
knn_best = knn_gs.best_estimator_
#check best n_neigbors value
print(knn_gs.best_params_)

{'n_neighbors': 14}


In [35]:
# Imprimir el tiempo de ejecución
end_time_1 = time.time()
print("Tiempo de ejecución:", end_time - start_time, "segundos")


Tiempo de ejecución: 31.754456520080566 segundos


**2.Random Forest**


Un bosque aleatorio se considera un modelo de conjunto en sí mismo, ya que es una colección de árboles de decisión combinados para hacer un modelo más preciso.

In [14]:
from sklearn.ensemble import RandomForestClassifier
#create a new random forest classifier
rf = RandomForestClassifier()
#create a dictionary of all values we want to test for n_estimators
params_rf = {'n_estimators': [50, 100, 200]}
#use gridsearch to test all values for n_estimators
rf_gs = GridSearchCV(rf, params_rf, cv=5)
#fit model to training data
rf_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [50, 100, 200]})

Crearemos un nuevo clasificador de bosque aleatorio y estableceremos los hiperparámetros que queremos ajustar. 'n_estimators' es el número de árboles en nuestro bosque aleatorio. Luego podemos ejecutar nuestra búsqueda de cuadrícula para encontrar la cantidad óptima de árboles.

Al igual que antes, guardaremos nuestro mejor modelo e imprimiremos el mejor valor de 'n_estimators'.

In [16]:
#save best model
rf_best = rf_gs.best_estimator_
#check best n_estimators value
print(rf_gs.best_params_)from sklearn.linear_model import LogisticRegression
#create a new logistic regression model
log_reg = LogisticRegression()
#fit the model to the training data
log_reg.fit(X_train, y_train)

{'n_estimators': 200}


**3. Logistic Regression**

Aunque tiene 'regresión' en su nombre, la regresión logística es un método de clasificación. Este es más simple ya que no ajustaremos ningún hiperparámetro. Solo necesitamos crear y entrenar el modelo.

In [17]:
from sklearn.linear_model import LogisticRegression
#create a new logistic regression model
log_reg = LogisticRegression()
#fit the model to the training data
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
#test the three models with the test data and print their accuracy scores
print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('rf: {}'.format(rf_best.score(X_test, y_test)))
print('log_reg: {}'.format(log_reg.score(X_test, y_test)))

knn: 0.7489177489177489
rf: 0.7575757575757576
log_reg: 0.7965367965367965


Como puede ver en el resultado, la regresión logística es la más precisa de las tres.

# **Voting Classifier**

Ahora que hemos construido nuestros tres modelos individuales, es hora de que construyamos nuestro clasificador de votación.

In [22]:
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
estimators=[('knn', knn_best), ('rf', rf_best), ('log_reg', log_reg)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

Primero, coloquemos nuestros tres modelos en una matriz llamada "estimadores". A continuación, crearemos nuestro clasificador de votos. Se necesitan dos entradas. El primero es nuestra matriz de estimadores de nuestros tres modelos. Estableceremos el parámetro de votación en duro, lo que le dice a nuestro clasificador que haga predicciones por mayoría de votos.

Ahora podemos ajustar nuestro modelo de conjunto a nuestros datos de entrenamiento y calificarlo en nuestros datos de prueba.

In [23]:
#fit model to training data
ensemble.fit(X_train, y_train)
#test our model on the test data
ensemble.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7835497835497836

# **Recursos**


https://towardsdatascience.com/ensemble-learning-using-scikit-learn-85c4531ff86a